# Tensorflow 공식 문서

### Import Tensorflow and Check version 2.0

In [1]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [2]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

### Load mnist datasets

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 채널 차원을 추가합니다.
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

### Using `tf.data`, shuffle datasets and make batch

In [4]:
# Train만 shuffle
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)).batch(32)

### Using keras' model subclassing API, generate `tf.keras` model.

In [5]:
class MyModel(Model):
    
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    
model = MyModel()

### Choose optimizer and loss function for training

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

### Choose metric for model. Output final results based on metrics collected during the epoch

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### Using `tf.GradientTape`, train model

In [8]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

### Test model

In [11]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = model(images)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [13]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)
        
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
        
    template = 'EPOCH: {}, LOSS: {}, ACCURACY: {}, TEST_LOSS: {}, TEST_ACCURACY: {}'
    print(template.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result()*100,
                          test_loss.result(),
                          test_accuracy.result()*100))

EPOCH: 1, LOSS: 0.07120607048273087, ACCURACY: 97.82222747802734, TEST_LOSS: 0.10000000149011612, TEST_ACCURACY: 98.19999694824219
EPOCH: 2, LOSS: 0.05694364011287689, ACCURACY: 98.2562484741211, TEST_LOSS: 0.10000000149011612, TEST_ACCURACY: 98.20999908447266
EPOCH: 3, LOSS: 0.047441061586141586, ACCURACY: 98.54066467285156, TEST_LOSS: 0.10000000149011612, TEST_ACCURACY: 98.29499816894531
EPOCH: 4, LOSS: 0.040639251470565796, ACCURACY: 98.74861145019531, TEST_LOSS: 0.10000000149011612, TEST_ACCURACY: 98.302001953125
EPOCH: 5, LOSS: 0.03567642346024513, ACCURACY: 98.90142822265625, TEST_LOSS: 0.10000000149011612, TEST_ACCURACY: 98.30999755859375


## Advanced

### Tensor
- 텐서는 다차원 배열입니다. 넘파이(NumPy) `ndarray` 객체와 비슷하며, `tf.Tensor` 객체는 데이터 타입과 크기를 가지고 있습니다. 또한 `tf.Tensor`는 GPU 같은 가속기 메모리에 상주할 수 있습니다. 텐서플로는 텐서를 생성하고 이용하는 풍부한 연산 라이브러리(tf.add, tf.matmul, tf.linalg.inv 등.)를 제공합니다. 이러한 연산은 자동으로 텐서를 파이썬 네이티브(native) 타입으로 변환합니다.

In [14]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

# 연산자 오버로딩(overloading) 또한 지원합니다.
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


- 각각의 `tf.Tensor`는 크기와 데이터 타입을 가지고 있습니다.

In [15]:
x = tf.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


넘파이 배열과 `tf.Tensor`의 가장 확연한 차이는 다음과 같습니다:

1. `텐서`는 가속기 메모리(GPU, TPU와 같은)에서 사용할 수 있습니다.
2. `텐서`는 불변성(immutable)을 가집니다.

## Numpy 호환성
텐서와 넘파이 배열 사이의 변환은 다소 간단합니다.

- 텐서플로 연산은 자동으로 넘파이 배열을 텐서로 변환합니다.
- 넘파이 연산은 자동으로 텐서를 넘파이 배열로 변환합니다.

텐서는 `.numpy()` 메서드(method)를 호출하여 넘파이 배열로 변환할 수 있습니다.<br>
가능한 경우, `tf.Tensor`와 배열은 메모리 표현을 공유하기 때문에 이러한 변환은 일반적으로 간단(저렴)합니다.<br>
그러나 `tf.Tensor`는 GPU 메모리에 저장될 수 있고, <br>
넘파이 배열은 항상 호스트 메모리에 저장되므로,<br>
이러한 변환이 항상 가능한 것은 아닙니다.<br>
**따라서 GPU에서 호스트 메모리로 복사가 필요합니다.**

In [17]:
import numpy as np

ndarray = np.ones([3, 3])

print("텐서플로 연산은 자동적으로 넘파이 배열을 텐서로 변환합니다.")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("\n그리고 넘파이 연산은 자동적으로 텐서를 넘파이 배열로 변환합니다.")
print(np.add(tensor, 1))

print("\n.numpy() 메서드는 텐서를 넘파이 배열로 변환합니다.")
print(tensor.numpy())

텐서플로 연산은 자동적으로 넘파이 배열을 텐서로 변환합니다.
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)

그리고 넘파이 연산은 자동적으로 텐서를 넘파이 배열로 변환합니다.
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]

.numpy() 메서드는 텐서를 넘파이 배열로 변환합니다.
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


## GPU 가속
대부분의 텐서플로 연산은 GPU를 사용하여 가속화됩니다. 어떠한 코드를 명시하지 않아도, 텐서플로는 연산을 위해 CPU 또는 GPU를 사용할 것인지를 자동으로 결정합니다. 필요시 텐서를 CPU와 GPU 메모리 사이에서 복사합니다. 연산에 의해 생성된 텐서는 전형적으로 연산이 실행된 장치의 메모리에 의해 실행됩니다. 예를 들어:

In [18]:
x = tf.random.uniform([3, 3])

print("GPU 사용이 가능한가 : "),
print(tf.test.is_gpu_available())

print("텐서가 GPU #0에 있는가 : "),
print(x.device.endswith('GPU:0'))

GPU 사용이 가능한가 : 
True
텐서가 GPU #0에 있는가 : 
True


## 장치 이름
`Tensor.device`는 텐서를 구성하고 있는 호스트 장치의 풀네임을 제공합니다. 이러한 이름은 프로그램이 실행중인 호스트의 네트워크 주소 및 해당 호스트 내의 장치와 같은 많은 세부 정보를 인코딩하며, 이것은 텐서플로 프로그램의 분산 실행에 필요합니다. 텐서가 호스트의 N번째 GPU에 놓여지면 문자열은 GPU:<N>으로 끝납니다.

## 명시적 장치 배치
텐서플로에서 "배치(replacement)"는 개별 연산을 실행하기 위해 장치에 할당(배치)하는 것입니다. 앞서 언급했듯이, 명시적 지침이 없을 경우 텐서플로는 연산을 실행하기 위한 장치를 자동으로 결정하고, 필요시 텐서를 장치에 복사합니다. 그러나 텐서플로 연산은 tf.device을 사용하여 특정한 장치에 명시적으로 배치할 수 있습니다. 예를 들어:

In [21]:
import time

def time_matmul(x):
    start = time.time()
    for loop in range(100):
        tf.matmul(x, x)

    result = time.time()-start

    print("100 loops: {:0.2f}ms".format(1000*result))

# CPU에서 강제 실행합니다.
print("On CPU:")
with tf.device("CPU:0"):
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("CPU:0")
    time_matmul(x)

# GPU #0가 이용가능시 GPU #0에서 강제 실행합니다.
if tf.test.is_gpu_available():
    print("On GPU:")
    with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
        x = tf.random.uniform([1000, 1000])
        assert x.device.endswith("GPU:0")
        time_matmul(x)

On CPU:
100 loops: 1025.22ms
On GPU:
100 loops: 3.00ms


## 데이터셋
이번에는 모델에 데이터를 제공하기 위한 파이프라인을 구축하기 위해 `tf.data.Dataset` API를 사용해볼 것입니다.

`tf.data.Dataset` API는 모델을 훈련시키고 평가 루프를 제공할, 간단하고 재사용 가능한 모듈로부터 복잡한 입력 파이프라인을 구축하기 위해 사용됩니다.

### 소스 데이터셋 생성
굉장히 유용한 함수중 하나인 `Dataset.from_tensors`, `Dataset.from_tensor_slices`와 같은 팩토리(factory) 함수 중 하나를 사용하거나 파일로부터 읽어들이는 객체인 `TextLineDataset` 또는 `TFRecordDataset`를 사용하여 소스 데이터셋을 생성하세요. 더 많은 정보를 위해서 텐서플로 데이터셋 가이드를 참조하세요.

In [45]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# CSV 파일 생성
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
    f.write(
        """Line 1
        Line 2
        Line 3"""
    )
ds_file = tf.data.TextLineDataset(filename)

### 변환 적용
맵(map), 배치(batch), 셔플(shuffle)과 같은 변환 함수를 사용하여 데이터셋의 레코드에 적용하세요.

In [46]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

### 반복
`tf.data.Dataset`은 레코드 순화를 지원하는 반복가능한 객체입니다.

In [47]:
print('ds_tensors 요소:')
for x in ds_tensors:
    print(' ', x)
    
print('\nds_file 요소:')
for x in ds_file:
    print(' ', x)

ds_tensors 요소:
  tf.Tensor([1 4], shape=(2,), dtype=int32)
  tf.Tensor([16 25], shape=(2,), dtype=int32)
  tf.Tensor([36  9], shape=(2,), dtype=int32)

ds_file 요소:
  tf.Tensor([b'Line 1' b'        Line 2'], shape=(2,), dtype=string)
  tf.Tensor([b'        Line 3'], shape=(1,), dtype=string)
